In [ ]:
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

prefix = '/mnt/data1/mimic/iii'
def dpath(fname):
    return os.path.join(prefix, fname)

def cleaned(fname):
    return os.path.join(prefix, 'cleaned', fname)

In [ ]:
uidcols = ['SUBJECT_ID', 'HADM_ID', 'CHARTTIME']

# Extract Progress Notes

In [ ]:
usecols = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'TEXT']
dtypes =  [int,          float,     object,      object,      str,        str,           float,     str]

notes = pd.read_csv(
    dpath('NOTEEVENTS.csv.gz'),
    usecols=usecols,
    dtype=dict(zip(usecols, dtypes)),
    parse_dates=['CHARTDATE', 'CHARTTIME'],
)
notes = notes.dropna(subset='HADM_ID')
notes['HADM_ID'] = notes['HADM_ID'].astype(int)

In [ ]:
progress_notes = notes[
    notes['CATEGORY'].isin({'Physician '}) &
    notes['DESCRIPTION'].str.contains('Physician|Attending|Fellow|Resident|PGY', case=False, regex=True) &
    notes['DESCRIPTION'].str.contains('prog', case=False) &
    notes['ISERROR'].isna()
]
progress_notes = progress_notes.sort_values(uidcols)

In [ ]:
print('Number of progress notes:', len(progress_notes))
print('Number of unique progress note chart times:', len(progress_notes.drop_duplicates(uidcols)))
print('Unique encounters with progress notes:', len(progress_notes['HADM_ID'].drop_duplicates()))
assert not progress_notes['CHARTTIME'].isna().any()

In [ ]:
progress_notes[uidcols + ['TEXT']].to_csv(cleaned('Progress_Notes.csv'), index=False)

# Load CHARTEVENTS

In [ ]:
_items = pd.read_csv(dpath('D_ITEMS.csv.gz'))
items = _items.dropna(axis=0, subset='LABEL')

parquet_file = dpath('CHARTEVENTS.parquet')
if not os.path.exists(parquet_file):
    chunksize = 1_000_000
    usecols = ['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE', 'VALUENUM', 'VALUEUOM']
    dtypes =  [int,          int,       int,      object,      str,     float,      str]

    chunks = []
    it = pd.read_csv(
        dpath('CHARTEVENTS.csv.gz'),
        chunksize=chunksize,
        usecols=usecols,
        dtype=dict(zip(usecols, dtypes)),
        parse_dates=['CHARTTIME'],
    )

    for chunk in tqdm(it):
        chunks.append(chunk)

    chart = pd.concat(chunks)

    _df = chart.merge(items[['ITEMID', 'LABEL', 'DBSOURCE']], how='left', on='ITEMID')
    _df = _df.sort_values(uidcols)
    _df.to_parquet(parquet_file)

df = pd.read_parquet(parquet_file)

In [ ]:
meas_count = df['ITEMID'].value_counts().reset_index().merge(items[['ITEMID', 'LABEL', 'DBSOURCE']], how='left', on='ITEMID')[['ITEMID', 'LABEL', 'DBSOURCE', 'count']]
def lookup(term, case=False, min_count=0):
    mask = meas_count['LABEL'].str.contains(term, case=case, regex=True)
    mask2 = meas_count['count'] >= min_count
    ret = meas_count.loc[mask & mask2]
    return ret

def values_of(val):
    return df[df['ITEMID'] == val]['VALUENUM'].value_counts()

def units_of(val):
    return df[df['ITEMID'] == val]['VALUEUOM'].value_counts()

def units(_df):
    return _df['VALUEUOM'].value_counts()

from collections.abc import Iterable
def filter(vals):
    if isinstance(vals, Iterable):
        vals = set(vals)
    else:
        vals = {vals}
    return df[df['ITEMID'].isin(vals)]

def clean(_df, name, asint=False, xlim=[0, 200]):
    if 'VALUEUOM' in _df.columns:
        print('=== Units ===')
        print(units(_df))
        print()
    if 'LABEL' in _df.columns:
        ax = sns.boxplot(_df, x='VALUENUM', y=_df['LABEL'] + ' (' + _df['ITEMID'].astype(str) + ')')
        ax.set_ylabel('LABEL (ITEMID)')
    else:
        ax = sns.boxplot(_df, x='VALUENUM')
    ax.set_xlim(xlim)
    ret = _df[uidcols + ['VALUENUM']].dropna(axis=0).rename(columns={'VALUENUM': name})
    ret = ret.groupby(uidcols)[name].mean()
    if asint:
        ret = ret.round().astype(int)
    ret = ret.reset_index()
    return ret

# Extract Vitals

### Glascow Coma Scale

In [ ]:
lookup('GCS')

In [ ]:
gcs_cv_itemid = 198
gcs_mv_itemids = {220739, 223900, 223901}

In [ ]:
gcs_cv_df = filter(gcs_cv_itemid)
gcs_mv_df = filter(gcs_mv_itemids)

In [ ]:
gcs_cv = gcs_cv_df.set_index(uidcols)['VALUENUM']
gcs_mv = pd.pivot(gcs_mv_df, index=uidcols, columns='LABEL', values='VALUENUM').sum(axis=1, skipna=False)

In [ ]:
gcs_df = pd.concat([gcs_cv, gcs_mv]).sort_index()
gcs_df.name = 'VALUENUM'
gcs_df = gcs_df.reset_index()

In [ ]:
gcs = clean(gcs_df, 'GCS', asint=True, xlim=[0, 20])

In [ ]:
gcs.to_csv(cleaned('GCS.csv'), index=False)

### Heart Rate

In [ ]:
lookup('heart rate|HR', min_count=10_000)

In [ ]:
hr_itemids = {211, 220045}
hr_df = filter(hr_itemids)

In [ ]:
hr = clean(hr_df, 'HR', asint=True, xlim=[-50, 250])

In [ ]:
hr.to_csv(cleaned('HR.csv'), index=False)

### Respiratory Rate

In [ ]:
rr_items = lookup('resp.*rate', min_count=10_000)
rr_items

In [ ]:
rr_itemids = set(rr_items['ITEMID'])
rr_df = filter(rr_itemids)

In [ ]:
rr = clean(rr_df, 'RR', asint=True, xlim=[-10, 200])

In [ ]:
rr.to_csv(cleaned('RR.csv'), index=False)

### Blood Pressure

In [ ]:
bp_item_df = lookup('blood pressure|bp', min_count=10_000)
bp_item_df = bp_item_df[
    (~bp_item_df['LABEL'].str.contains('alarm', case=False)) &
    (~bp_item_df['LABEL'].str.contains('IABP', case=False)) &
    (~bp_item_df['LABEL'].str.contains('rate', case=False)) &
    (~bp_item_df['LABEL'].str.contains('BP UAC', case=True)) &
    (~bp_item_df['LABEL'].str.contains('BP PAL', case=True)) &
    (~bp_item_df['LABEL'].str.contains('BP Cuff', case=True)) # weird range and weird units (cc/min)?
]
bp_item_df

In [ ]:
sbp_itemids = [51,   220050]
dbp_itemids = [8368, 220051]
# mbp_itemids = [52,   220052]

snbp_itemids = [455,  220179]
dnbp_itemids = [8441, 220180]
# mnbp_itemids = [456,  220181]

In [ ]:
sbp_df = filter(sbp_itemids)
dbp_df = filter(dbp_itemids)

snbp_df = filter(snbp_itemids)
dnbp_df = filter(dnbp_itemids)

In [ ]:
sbp = clean(sbp_df, 'VALUENUM', asint=True)

In [ ]:
dbp = clean(dbp_df, 'VALUENUM', asint=True)

In [ ]:
snbp = clean(snbp_df, 'VALUENUM', asint=True)

In [ ]:
dnbp = clean(dnbp_df, 'VALUENUM', asint=True)

In [ ]:
bp_df = pd.concat([sbp, dbp])
bp_df['LABEL'] = ['SBP'] * len(sbp) + ['DBP'] * len(dbp)

nbp_df = pd.concat([snbp, dnbp])
nbp_df['LABEL'] = ['SBP'] * len(snbp) + ['DBP'] * len(dnbp)

In [ ]:
bp_pivot = pd.pivot(bp_df, index=uidcols, columns='LABEL', values='VALUENUM')
nbp_pivot = pd.pivot(nbp_df, index=uidcols, columns='LABEL', values='VALUENUM')

In [ ]:
bp = bp_pivot.dropna(axis=0, subset=['DBP', 'SBP']).astype(int).reset_index()
bp.columns.name = None

nbp = nbp_pivot.dropna(axis=0, subset=['DBP', 'SBP']).astype(int).reset_index()
nbp.columns.name = None

In [ ]:
bp.to_csv(cleaned('BP.csv'), index=False)
nbp.to_csv(cleaned('NBP.csv'), index=False)

### SpO2

In [ ]:
lookup('pulse.*ox|spo2')

In [ ]:
spo2_itemids = {220277, 646}
spo2_df = filter(spo2_itemids)

In [ ]:
spo2 = clean(spo2_df, 'SpO2', asint=True)

In [ ]:
spo2.to_csv(cleaned('SpO2.csv'), index=False)

# Extract Labs

### HCO3

In [ ]:
co2_items = lookup('HCO3|CO2|Bicarb|Carbon', min_count=1000)
co2_items

In [ ]:
filter(co2_items['ITEMID'])

In [ ]:
filter(777)

In [ ]:
co2_itemids = {227443, 787, 3810, 3808, 225698}
co2_df = filter(co2_itemids)

In [ ]:
ax = sns.boxplot(co2_df, x='VALUENUM', y='LABEL')
ax.set_xlim([0, 100])

In [ ]:
co2_df['VALUEUOM'].value_counts()

In [ ]:
co2 = clean(co2_df, 'HCO3')
co2

In [ ]:
co2.to_csv(dpath('HCO3.csv'), index=False)

### Cl

In [ ]:
lookup('chlor')

In [ ]:
lookup('Cl')

In [ ]:
cl_itemids = {788, 220602, 1523, 3747, 226536}
cl_df = filter(cl_itemids)

In [ ]:
ax = sns.boxplot(cl_df, x='VALUENUM', y='LABEL')
ax.set_xlim([0, 200])

In [ ]:
units(cl_df)

In [ ]:
cl = clean(cl_df, 'Cl')
cl

In [ ]:
cl.to_csv(dpath('Cl.csv'), index=False)

### K

### Na

### Anion Gap

### Ca

### Glucose

### Creatinine

### eGFR

### BUN

### Total Protein

### Albumin

### Total Bilirubin

### Conjugated Bilirubin

### Unconjugated Bilirubin

### Alk Phos

### AST

### ALT

# Extract Progress Notes

In [ ]:
usecols = ['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'CATEGORY', 'DESCRIPTION', 'ISERROR', 'TEXT']
dtypes =  [int,          float,     object,      object,      str,        str,           float,     str]

notes = pd.read_csv(
    dpath('NOTEEVENTS.csv.gz'),
    usecols=usecols,
    dtype=dict(zip(usecols, dtypes)),
    parse_dates=['CHARTDATE', 'CHARTTIME'],
)

In [ ]:
notes = notes.dropna(axis=0, subset=['HADM_ID'])
notes['HADM_ID'] = notes['HADM_ID'].astype(int)

In [ ]:
note_types = {
    'Attendinding Progress Note',
    'Attending / Resident Progress Note',
    'Attending Daily Progress Note',
    'Attending Progress Note',
    'Attending Progress Note - MICU',
    'Attending Progress note',
    'Attending progress note',
    'Attending/Physician Resident Progress Note - MICU',
    'Attg Progress Note',
    'CCU Progress Note',
    'CCU Resident Daily Progress Note',
    'CCU Resident Progress Note',
    'CCU Staff Progress Note',
    'CCU progress note',
    'CCU resident progress note',
    'Cardiology Resident Progress Note',
    'Cardiology Resident Progress Note - CCU',
    'Cardiology Weekend Consult Attending Progress Note',
    'Cardiology Weekend Consult Progress Note',
    'Cardiology- Electrophysiology Progress Note',
    'Critical care attending Progress note',
    'Critical care attending progress note',
    'Critical care progress note',
    'Daily Progress Note',
    'EP Fellow Progress Note',
    'EP Progress Note',
    'EPS Progress Note',
    'Electrophysiology Consult Progress Note',
    'FICU resident progress note',
    'FICU resident progress note/MICU staff addendum',
    'Felllow / Physician Attending Progress Note - MICU',
    'Fellow / Physician Attending Progress Note - MICU',
    'Fellow Progress Note',
    'Fellow Progress Note - MICU',
    'Fellow/Attending Progress Note - MICU',
    'GI  Consult Progress Note',
    'ICU - Resident Progress Note',
    'ICU Atending Progress note',
    'ICU Attending Progress Note',
    'ICU Attending Progress Note - family meeting',
    'ICU Attending Progress Report',
    'ICU Attending/Fellow Progress Note - MICU',
    'ICU Fellow Progress Note - MICU',
    'ICU Fellow Progress Note - MICU atending addendum',
    'ICU Fellow Progress Note - MICU attedning addendum',
    'ICU Fellow Progress Note - MICU attending addendum',
    'ICU Fellow Progress Note - attending addendum MICU',
    'ICU Fellow Progress Note - attending addendum-MICU',
    'ICU Fellow Progress Note -attending addendum- MICU',
    'ICU Fellow Progress Note -attending addendum-MICU',
    'ICU Fellow Progress Note /attending addendum- MICU',
    'ICU Fellow/Attending Progress Note - MICU',
    'ICU Physican Progress Note',
    'ICU Physician Progress Note',
    'ICU Physician Progress Note -- Attending and Resid',
    'ICU Progress Note',
    'ICU Resident Progress Note',
    'MICU  staff progress note',
    'MICU - Resident Progress Note',
    'MICU - Resident/Attending Progress Note',
    'MICU Attending / Resident Progress Note',
    'MICU Attending Physician Progress Note',
    'MICU Attending Progress / Family Meeting Note',
    'MICU Attending Progress NOte',
    'MICU Attending Progress Note',
    'MICU Attending Progress Note - Family Meeting',
    'MICU Fellow Progress Note',
    'MICU Fellow Progress note',
    'MICU Fellow/Attending Progess Note',
    'MICU Fellow/Attending Progress Note',
    'MICU MS-IV/Attending Progress Note',
    'MICU Physician Progress Note',
    'MICU Resident / Attending Progress Note',
    'MICU Resident Daily Progress Note',
    'MICU Resident Progres Note',
    'MICU Resident Progress Note',
    'MICU Resident Progress note',
    'MICU Resident progress note',
    'MICU Resident/Attending Daily Progress Note',
    'MICU Resident/Attending Progress Note',
    'MICU attending Progress Note',
    'MICU attending addendum/progress note',
    'MICU attending progress note',
    'MICU resident physician progress note',
    'MICU resident progress note',
    'MICU staff Progress Note',
    'MICU staff progress note',
    'MS4 Progress Note',
    'Med Student Progress Note - CCU',
    'Medical Student CCU Progress Note',
    'Medical Student Progress Note',
    'Medical Student Progress Note - CCU',
    'Medical Student Progress Note - MICU',
    'Medical Student Progress Note CCU',
    'Medical Student/Attending Progress Note - MICU',
    'Medical student Progress Note - MICU',
    'Micu staff progress note',
    'PGY 1 progress note',
    'PGY-1 Progress Note',
    'PGY1 - Daily Progress Note',
    'PGY1 - progress note MICU',
    'PGY1 CCU Team Progress Note',
    'PGY1 Progress Note',
    'PGY1 Progress note',
    'PGY1 Resident Progress note',
    'PGY1- Daily Progress Note, MICU',
    'PGY1/Attending Daily Progress note MICU',
    'PGYI progess note',
    'PGYI progress note',
    'PROGRESS NOTE',
    'PROGRESS NOTE - RESIDENT',
    'Physican Resident Progress Note',
    'Physicial Resident Progress Note - MICU',
    'Physician Attending  Progress Note',
    'Physician Attending / Resident Progress Note',
    'Physician Attending Fellow Progress Note - MICU',
    'Physician Attending Progress Note',
    'Physician Attending and Fellow Progress Note - MIC',
    'Physician Attending and fellow Progress Note - MIC',
    'Physician Attending/ Resident Progress Note - MICU',
    'Physician Attending/Fellow Progress Note - MICU',
    'Physician Attending/Resident Progress Note',
    'Physician Attending/Resident Progress Note - CCU',
    'Physician Attending/Resident Progress Note - MICU',
    'Physician Consult Progress Note',
    'Physician Felllow / Attending Progress Note - MICU',
    'Physician Felllow/Attending Progress Note - MICU',
    'Physician Fellow /  Attending Progress Note - MICU',
    'Physician Fellow / Attending Progress Note - MICU',
    'Physician Fellow Progress Note',
    'Physician Fellow Progress Note - MICU',
    'Physician Fellow Progress Note - MICU attending ad',
    'Physician Fellow Progress Note - MICU attending pr',
    'Physician Fellow Progress Note -MICU attending add',
    'Physician Fellow Progress Note -MICU attending not',
    'Physician Fellow Progress Note -attending addendum',
    'Physician Fellow Progress Note /attending addendum',
    'Physician Fellow Progress Note-MICU attending adde',
    'Physician Fellow Progress Note/Attendign progress',
    'Physician Fellow Progress Note/MICU attending adde',
    'Physician Fellow and Attending Progress Note - MIC',
    'Physician Fellow/ Attending Progress Note - MICU',
    'Physician Fellow/Attending Progress Note - MICU',
    'Physician Felow/Attending Progress Note - MICU',
    'Physician Medical Student Progress Note - MICU',
    'Physician Medical Student/Attending Progress Note',
    'Physician Progress',
    'Physician Progress Note',
    'Physician Progress Note - CCU',
    'Physician Progress Note - MICU',
    'Physician Progress Note - Resident and Attending',
    'Physician Progress Note CCU',
    'Physician Progress Note-CCU',
    'Physician Residen Progress Note',
    'Physician Residen/Attending Progress Note - MICU',
    'Physician Resident / Attending Progress Note',
    'Physician Resident / Attending Progress Note - MIC',
    'Physician Resident / attending Progress Note - MIC',
    'Physician Resident /Attending Progress Note',
    'Physician Resident /Attending Progress Note - MICU',
    'Physician Resident /attending Progress Note - MICU',
    'Physician Resident Admission/Progress Note - CCU',
    'Physician Resident MICU Progression Note',
    'Physician Resident Progess Note',
    'Physician Resident Progress Note',
    'Physician Resident and  MICU attending --MICU prog',
    'Physician Resident and Attending  Progress Note',
    'Physician Resident and Attending Progress Note',
    'Physician Resident and Attending Progress Note - M',
    'Physician Resident and Attg Progress Note - MICU',
    'Physician Resident and attending  Progress Note',
    'Physician Resident and attending  Progress Note -',
    'Physician Resident and attending Progress Note',
    'Physician Resident attending  Progress Note - MICU',
    'Physician Resident-Attending Progress Note',
    'Physician Resident/ Attending  Progress Note',
    'Physician Resident/ attending Progress Note - MICU',
    'Physician Resident/Atending Progress Note - MICU',
    'Physician Resident/Attending  Progress Note',
    'Physician Resident/Attending  Progress Note - MICU',
    'Physician Resident/Attending Progess Note',
    'Physician Resident/Attending Progress Note',
    'Physician Resident/Attending Progress Note - CCU',
    'Physician Resident/Attending Progress Note - MICU',
    'Physician Resident/Attending Progress Note - MICU`',
    'Physician Resident/Attending Progress Note - MICUf',
    'Physician Resident/attending  Progress Note - MICU',
    'Physician Resident/attending Progress Note - MICU',
    'Physician Residentand Attending  Progress Note',
    'Physician Residentand attending  Progress Note',
    'Physician Residentand attending  Progress Note - M',
    'Physician Surgical Progress Note',
    'Physician fellow Progress Note',
    'Physician progress note',
    'Progress Note',
    'Progress Note - Resident',
    'Progress Note CCU',
    'Progress Notes',
    'Progress note',
    'Progress note - Resident',
    'RESIDENT MICU PROGRESS NOTE',
    'RESIDENT PROGRESS NOTE - CCU',
    'RESIDENT PROGRESS NOTE-CCU',
    'Resident / Attending Progress Note',
    'Resident / Attending Progress Notes',
    'Resident Daily Progress Note',
    'Resident Daily Progress note',
    'Resident MICU Progress Note',
    'Resident MICU progress note',
    'Resident Physician Progress Note',
    'Resident Physician Progress Note - CCU',
    'Resident Physician Progress Note - MICU',
    'Resident Physician Progress Note-CCU',
    'Resident Progess Note',
    'Resident Progress Not - MICU',
    'Resident Progress Note',
    'Resident Progress Note - CCU',
    'Resident Progress Note - MICU',
    'Resident Progress Note / Attending Addendum',
    'Resident Progress Note CCU',
    'Resident Progress Note CCu',
    'Resident Progress Note MICU',
    'Resident Progress Note, CCU',
    'Resident Progress Note, CCu',
    'Resident Progress Note- CCU',
    'Resident Progress Note- MICU',
    'Resident Progress Note-MICU',
    'Resident Progress Note/Attending Addendum',
    'Resident Progress note',
    'Resident progress Note',
    'Resident progress note',
    'Resident progress note - TSICU',
    'Resident progress note ICU',
    'Resident/Attending Progress Note',
    'Resident/Attending Progress Note - MICU',
    'Surgery Progress note',
    'TRICU resident progress note',
    'TSICU progress note',
    'crit care staff progress note',
    'critical care Progress note',
    'critical care attending progress note',
    'critical care progress note',
    'micu attending progress note',
    'progress note',
}

In [ ]:
note_cats = {
    'Physician ', # beware the trailing space
    'General',
    'Consult',
}

In [ ]:
prog_notes = notes.loc[(notes['DESCRIPTION'].isin(note_types)) & (notes['CATEGORY'].isin(note_cats))]
assert not prog_notes['CHARTTIME'].isna().any()
prog_notes = prog_notes[prog_notes['ISERROR'] != 1]
prog_notes = prog_notes[['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'TEXT']]
prog_notes = prog_notes.sort_values(['SUBJECT_ID', 'HADM_ID', 'CHARTTIME'])

In [ ]:
prog_notes

In [ ]:
prog_notes.to_csv(dpath('Progress_Notes.csv'), index=False)

In [ ]:
prog_notes['HADM_ID'].drop_duplicates()

In [ ]:
prog_notes['SUBJECT_ID'].drop_duplicates()